# Vaccination Progress in the US and World

<h2> What is the COVID-19/Coronavirus? </h2>

* According to the CDC, "coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus. Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.  Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness."
* Continuing, "the best way to prevent and slow down transmission is to be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face."
* Stay informed about COVID-19. Practice social distancing and healthy practices (washing hands). Educate your friends and family on the dangers of COVID-19. 
* Yes, there's currently a vaccine out, but the vaccine will take quite a while to distribute to everyone. Thus, I encourage everyone reading to be patient about COVID so we can unite together and minimize the catastrophic damage the coronavirus has inflicted.

<h2> So what role do vaccines play? </h2>

* How well it works: 95% efficacy in preventing COVID-19 in those without prior infection. In clinical trials, the vaccine was 100% effective at preventing severe disease.
* In early May, the Pfizer-BioNTech vaccine was found to be more than 95% effective against severe disease or death from the Alpha variant (first detected in the United Kingdom) and the Beta variant (first identified in South Africa) in two studies based on real-world use of the vaccine. As far as the Delta variant (first seen in India), two studies reported by Public Health England that have not yet been peer reviewed showed that full vaccination (after two doses) is 88% effective against symptomatic disease and 96% effective against hospitalization.

Bottom line: Vaccines are **very** effective, so if you haven't already and have the ability to, <span style="color: green">please get vaccinated</span>! To learn more about COVID vaccines, click <a href="https://www.yalemedicine.org/news/covid-19-vaccine-comparison">here</a>, and to learn why we need more people to be vaccinated, learn about herd immunity <a href="https://www.mayoclinic.org/diseases-conditions/coronavirus/in-depth/herd-immunity-and-coronavirus/art-20486808">here</a>. Also feel free to check out my "Daily COVID-19 Interactive Plots" Kaggle notebook <a href="https://www.kaggle.com/ironicninja/daily-covid-19-interactive-plots">here</a>, I promise you'll find alot of good stuff there.

<h2> Table of Contents </h2>
<ol style="font-size: 16px">
    <li><a href="https://www.kaggle.com/ironicninja/vaccination-progress-us-world#United-States-Vaccination-Progress">United States Vaccination Progress</a></li>
    <li><a href="https://www.kaggle.com/ironicninja/vaccination-progress-us-world#Countries-Vaccination-Progress">Countries Vaccination Progress</a></li>
</ol>

<h2> Visualizations Included </h2> 
<ol style="font-size: 16px">
    <li>Total vaccinations given</li>
    <li>Total people vaccinated</li>
    <li>Percentage of people vaccinated by state/country</li>
    <li>Daily number of vaccinations</li>
    <li>Days left until everyone is fully vaccinated</li>
    <li>Individual progress by states/countries</li>
 </ol>

I may add some analysis in the future (vaccination progress compared to deaths, etc.), so if you like this work, please <span style="color: green">upvote & comment</span>! I would really appreciate it 😊.

All data is collected from Our World in Data (from the original raw github files).

<h2> Essential Imports </h2>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

# United States Vaccination Progress

One important note is that the <a href="https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv">data</a> I use does NOT include COVID data for New York.

In [ ]:
now = datetime.utcnow() - timedelta(hours=19)
today = now.strftime("%Y-%m-%d")
print(f"Today's date is {today}.")

In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

Initialize the data:

In [ ]:
original_states_df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv")
original_states_df['location'] = original_states_df['location'].map(us_state_abbrev) # e.g., Alaska -> AK
original_states_df.dropna(subset=['location'], inplace=True) # drop any location which isn't in the abbreviation dict
original_states_df.reset_index(drop=True, inplace=True)
original_states_df.reset_index(inplace=True)
for loc in us_state_abbrev.values():
    if loc == 'NY':
        continue
    loc_ser = original_states_df.loc[original_states_df['location'] == loc].iloc[0]
    index = loc_ser['index']
    original_states_df.loc[index] = loc_ser.fillna(0)

original_states_df.fillna(method="ffill", inplace=True)
original_states_df.fillna(0, inplace=True)

Add variables (all pretty self-explanatory):

In [ ]:
original_states_df['unused_vaccines'] = original_states_df['total_distributed'] - original_states_df['total_vaccinations']
original_states_df['one_dose'] = original_states_df['people_vaccinated'] - original_states_df['people_fully_vaccinated']
original_states_df['one_dose_per_hundred'] = (original_states_df['people_vaccinated_per_hundred'] - original_states_df['people_fully_vaccinated_per_hundred']).round(2)
original_states_df['population'] = (original_states_df['people_fully_vaccinated']/original_states_df['people_fully_vaccinated_per_hundred'].replace(0, 1)*100).astype(int)
original_states_df['how_many_more_vaccines'] = (2*(original_states_df['population'] - original_states_df['people_vaccinated']) + original_states_df['one_dose']).astype(int)
original_states_df['days_until_done'] = (original_states_df['how_many_more_vaccines']/original_states_df['daily_vaccinations'].replace(0, 1)).round(2)
original_states_df.head()

Create another DataFrame with a subset of the original data (only data for today):

In [ ]:
states_df = original_states_df.loc[original_states_df['date'] == today]
states_df.head()

<h3> Figure 1: Total Vaccinations Given in the United States </h3>

Visualization of raw numbers for vaccinations given in the United States. Also includes data on total distributed vaccines and unused vaccines (calculated as the difference between total distributed and total vaccinations given).

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=states_df['location'],
    z=states_df['total_vaccinations'].astype(int),
    locationmode='USA-states',
    colorscale="Reds",
    colorbar_title="Total Vaccinations Given",
    text='Total Distributed: ' + states_df['total_distributed'].astype(int).apply('{:,}'.format) + 
        '<br>Unused Vaccines: ' + states_df['unused_vaccines'].astype(int).apply('{:,}'.format)
))

fig.update_layout(
    geo_scope='usa',
    title={'text': f"Total Vaccinations Given in the United States on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Figure 2: Total People Vaccinated in the United States </h3>

Visualization of the total number of people vaccinated in the United States. Also includes data on people fully vaccinated and people who only have one dose of a vaccine (calculated as people vaccinated minus people fully vaccinated).

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=states_df['location'],
    z=states_df['people_vaccinated'].astype(int),
    locationmode='USA-states',
    colorscale="Reds",
    colorbar_title="Total People Vaccinated",
    text='Fully Vaccinated: ' + states_df['people_fully_vaccinated'].astype(int).apply('{:,}'.format) + 
        '<br>Only One Dose: ' + states_df['one_dose'].astype(int).apply('{:,}'.format)
))

fig.update_layout(
    geo_scope='usa',
    title={'text': f"Total People Vaccinated in the United States on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Figure 3: Percentage of People Vaccinated by State </h3>

Visualization of the percentage of people vaccinated by state. Also includes data on the percentage of people fully vaccinated and the percentage of people with only one dose by state (calculated as the percentage of people vaccinated minus the percentage of people fully vaccinated).

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=states_df['location'],
    z=states_df['people_vaccinated_per_hundred'].astype(float),
    locationmode='USA-states',
    colorscale="Reds",
    colorbar_title="Percentage Vaccinated",
    text='Fully Vaccinated Percentage: ' + states_df['people_fully_vaccinated_per_hundred'].apply('{:,}'.format) + '%' + 
        '<br>Only One Dose Percentage: ' + states_df['one_dose_per_hundred'].apply('{:,}'.format) + '%'
))

fig.update_layout(
    geo_scope='usa',
    title={'text': f"Percentage of People Vaccinated by State in the United States on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Figure 4. Daily Vaccinations in the United States </h3>

Visualization of the rate vaccines are being given in the United States at this time.

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=states_df['location'],
    z=states_df['daily_vaccinations_per_million'].astype(float),
    locationmode='USA-states',
    colorscale="Reds",
    colorbar_title="Per Million",
    text='Total Daily Vaccinations: ' + states_df['daily_vaccinations'].astype(int).apply('{:,}'.format) 
))

fig.update_layout(
    geo_scope='usa',
    title={'text': f"Daily Vaccinations Per Million in the United States on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Fig 5. Days Left Until Everyone is Fully Vaccinated in the United States </h3>

Visualization of how many days are left until everyone is fully vaccinated. Also includes data on how many total vaccinations are needed and the population of the state.

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=states_df['location'],
    z=states_df['days_until_done'].astype(float),
    locationmode='USA-states',
    colorscale="Reds",
    colorbar_title="Days Left",
    text='Total Vaccinations Needed: ' + states_df['how_many_more_vaccines'].astype(int).apply('{:,}'.format) +
    '<br>Population: ' + states_df['population'].astype(int).apply('{:,}'.format)
))

fig.update_layout(
    geo_scope='usa',
    title={'text': f"Days Left Until Fully Vaccinated in the United States on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Figure 6. Progress by State </h3>

Same visualizations as above except you can look at the progress/trend each state has been making. Default is to look 30 days back. Change ```days_prev``` in the ```state_visualization``` function to look a different number of days back.

In [ ]:
def state_visualization(var, title, yaxis = 'Percentage', days_prev = 30):
    c = 0
    buttons_list = []
    base_list = [False for i in range(len(states_df))]
    for i in states_df.index:
        tmp_list = base_list.copy()
        tmp_list[c] = True
        buttons_list.append(dict(
            args=[{"visible": tmp_list}],
            label=states_df.loc[i]['location'],
            method="update"
        ))
        c += 1

    c = 0
    fig = go.Figure()
    for i in states_df.index:
        location_name = states_df.loc[i]['location']
        prev_df = original_states_df.loc[original_states_df['location'] == location_name][-days_prev:]
        fig.add_trace(go.Scatter(name=location_name, x=prev_df["date"], y=prev_df[var], visible=False if c != 0 else True))
        c += 1

    fig.update_layout(
        updatemenus=[
            dict(
                buttons=buttons_list,
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )

    fig.update_layout(title={'text': f'{title} - State, Past {days_prev} Days', 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, showlegend=False, yaxis_title=f"{yaxis}")
    fig.show()

In [ ]:
state_visualization('people_vaccinated_per_hundred', 'Percentage of People Vaccinated')

In [ ]:
state_visualization('people_fully_vaccinated_per_hundred', 'Percentage of People Fully Vaccinated')

In [ ]:
state_visualization('daily_vaccinations', 'Number of Daily Vaccinations', yaxis="Per Million")

In [ ]:
state_visualization('how_many_more_vaccines', 'How Many More Vaccines are Needed?', yaxis="Total Vaccines")

In [ ]:
state_visualization('days_until_done', 'Days Left Until Fully Vaccinated', yaxis="Days Left")

# Countries Vaccination Progress

In [ ]:
country_abbrev = {
'Afghanistan': 'AFG',
 'Albania': 'ALB',
 'Algeria': 'DZA',
 'American Samoa': 'ASM',
 'Andorra': 'AND',
 'Angola': 'AGO',
 'Anguilla': 'AIA',
 'Antigua and Barbuda': 'ATG',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Aruba': 'ABW',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Bahamas, The': 'BHM',
 'Bahrain': 'BHR',
 'Bangladesh': 'BGD',
 'Barbados': 'BRB',
 'Belarus': 'BLR',
 'Belgium': 'BEL',
 'Belize': 'BLZ',
 'Benin': 'BEN',
 'Bermuda': 'BMU',
 'Bhutan': 'BTN',
 'Bolivia': 'BOL',
 'Bosnia and Herzegovina': 'BIH',
 'Botswana': 'BWA',
 'Brazil': 'BRA',
 'British Virgin Islands': 'VGB',
 'Brunei': 'BRN',
 'Bulgaria': 'BGR',
 'Burkina Faso': 'BFA',
 'Burma': 'MMR',
 'Burundi': 'BDI',
 'Cabo Verde': 'CPV',
 'Cambodia': 'KHM',
 'Cameroon': 'CMR',
 'Canada': 'CAN',
 'Cayman Islands': 'CYM',
 'Central African Republic': 'CAF',
 'Chad': 'TCD',
 'Chile': 'CHL',
 'China': 'CHN',
 'Colombia': 'COL',
 'Comoros': 'COM',
 'Congo, Democratic Republic of the': 'COD',
 'Congo, Republic of the': 'COG',
 'Cook Islands': 'COK',
 'Costa Rica': 'CRI',
 "Cote d'Ivoire": 'CIV',
 'Croatia': 'HRV',
 'Cuba': 'CUB',
 'Curacao': 'CUW',
 'Cyprus': 'CYP',
 'Czech Republic': 'CZE',
 'Denmark': 'DNK',
 'Djibouti': 'DJI',
 'Dominica': 'DMA',
 'Dominican Republic': 'DOM',
 'Ecuador': 'ECU',
 'Egypt': 'EGY',
 'El Salvador': 'SLV',
 'Equatorial Guinea': 'GNQ',
 'Eritrea': 'ERI',
 'Estonia': 'EST',
 'Ethiopia': 'ETH',
 'Falkland Islands (Islas Malvinas)': 'FLK',
 'Faroe Islands': 'FRO',
 'Fiji': 'FJI',
 'Finland': 'FIN',
 'France': 'FRA',
 'French Polynesia': 'PYF',
 'Gabon': 'GAB',
 'Gambia, The': 'GMB',
 'Georgia': 'GEO',
 'Germany': 'DEU',
 'Ghana': 'GHA',
 'Gibraltar': 'GIB',
 'Greece': 'GRC',
 'Greenland': 'GRL',
 'Grenada': 'GRD',
 'Guam': 'GUM',
 'Guatemala': 'GTM',
 'Guernsey': 'GGY',
 'Guinea-Bissau': 'GNB',
 'Guinea': 'GIN',
 'Guyana': 'GUY',
 'Haiti': 'HTI',
 'Honduras': 'HND',
 'Hong Kong': 'HKG',
 'Hungary': 'HUN',
 'Iceland': 'ISL',
 'India': 'IND',
 'Indonesia': 'IDN',
 'Iran': 'IRN',
 'Iraq': 'IRQ',
 'Ireland': 'IRL',
 'Isle of Man': 'IMN',
 'Israel': 'ISR',
 'Italy': 'ITA',
 'Jamaica': 'JAM',
 'Japan': 'JPN',
 'Jersey': 'JEY',
 'Jordan': 'JOR',
 'Kazakhstan': 'KAZ',
 'Kenya': 'KEN',
 'Kiribati': 'KIR',
 'Korea, North': 'PRK',
 'Korea, South': 'KOR',
 'Kosovo': 'KSV',
 'Kuwait': 'KWT',
 'Kyrgyzstan': 'KGZ',
 'Laos': 'LAO',
 'Latvia': 'LVA',
 'Lebanon': 'LBN',
 'Lesotho': 'LSO',
 'Liberia': 'LBR',
 'Libya': 'LBY',
 'Liechtenstein': 'LIE',
 'Lithuania': 'LTU',
 'Luxembourg': 'LUX',
 'Macau': 'MAC',
 'Macedonia': 'MKD',
 'Madagascar': 'MDG',
 'Malawi': 'MWI',
 'Malaysia': 'MYS',
 'Maldives': 'MDV',
 'Mali': 'MLI',
 'Malta': 'MLT',
 'Marshall Islands': 'MHL',
 'Mauritania': 'MRT',
 'Mauritius': 'MUS',
 'Mexico': 'MEX',
 'Micronesia, Federated States of': 'FSM',
 'Moldova': 'MDA',
 'Monaco': 'MCO',
 'Mongolia': 'MNG',
 'Montenegro': 'MNE',
 'Morocco': 'MAR',
 'Mozambique': 'MOZ',
 'Namibia': 'NAM',
 'Nepal': 'NPL',
 'Netherlands': 'NLD',
 'New Caledonia': 'NCL',
 'New Zealand': 'NZL',
 'Nicaragua': 'NIC',
 'Nigeria': 'NGA',
 'Niger': 'NER',
 'Niue': 'NIU',
 'Northern Mariana Islands': 'MNP',
 'Norway': 'NOR',
 'Oman': 'OMN',
 'Pakistan': 'PAK',
 'Palau': 'PLW',
 'Panama': 'PAN',
 'Papua New Guinea': 'PNG',
 'Paraguay': 'PRY',
 'Peru': 'PER',
 'Philippines': 'PHL',
 'Poland': 'POL',
 'Portugal': 'PRT',
 'Puerto Rico': 'PRI',
 'Qatar': 'QAT',
 'Romania': 'ROU',
 'Russia': 'RUS',
 'Rwanda': 'RWA',
 'Saint Kitts and Nevis': 'KNA',
 'Saint Lucia': 'LCA',
 'Saint Martin': 'MAF',
 'Saint Pierre and Miquelon': 'SPM',
 'Saint Vincent and the Grenadines': 'VCT',
 'Samoa': 'WSM',
 'San Marino': 'SMR',
 'Sao Tome and Principe': 'STP',
 'Saudi Arabia': 'SAU',
 'Senegal': 'SEN',
 'Serbia': 'SRB',
 'Seychelles': 'SYC',
 'Sierra Leone': 'SLE',
 'Singapore': 'SGP',
 'Sint Maarten': 'SXM',
 'Slovakia': 'SVK',
 'Slovenia': 'SVN',
 'Solomon Islands': 'SLB',
 'Somalia': 'SOM',
 'South Africa': 'ZAF',
 'South Sudan': 'SSD',
 'Spain': 'ESP',
 'Sri Lanka': 'LKA',
 'Sudan': 'SDN',
 'Suriname': 'SUR',
 'Swaziland': 'SWZ',
 'Sweden': 'SWE',
 'Switzerland': 'CHE',
 'Syria': 'SYR',
 'Taiwan': 'TWN',
 'Tajikistan': 'TJK',
 'Tanzania': 'TZA',
 'Thailand': 'THA',
 'Timor-Leste': 'TLS',
 'Togo': 'TGO',
 'Tonga': 'TON',
 'Trinidad and Tobago': 'TTO',
 'Tunisia': 'TUN',
 'Turkey': 'TUR',
 'Turkmenistan': 'TKM',
 'Tuvalu': 'TUV',
 'Uganda': 'UGA',
 'Ukraine': 'UKR',
 'United Arab Emirates': 'ARE',
 'United Kingdom': 'GBR',
 'United States': 'USA',
 'Uruguay': 'URY',
 'Uzbekistan': 'UZB',
 'Vanuatu': 'VUT',
 'Venezuela': 'VEN',
 'Vietnam': 'VNM',
 'Virgin Islands': 'VGB',
 'West Bank': 'WBG',
 'Yemen': 'YEM',
 'Zambia': 'ZMB',
 'Zimbabwe': 'ZWE'
}

Initialize the data:

In [ ]:
days_ago = (now - timedelta(days=7)).strftime("%Y-%m-%d")
original_country_df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv")
original_country_df = original_country_df.loc[~original_country_df['iso_code'].str.contains('OWID')]
original_country_df.reset_index(drop=True, inplace=True)
original_country_df.reset_index(inplace=True)
for loc in original_country_df['location'].unique():
    loc_ser = original_country_df.loc[original_country_df['location'] == loc].iloc[0]
    index = loc_ser['index']
    original_country_df.loc[index] = loc_ser.fillna(0)

original_country_df.fillna(method="ffill", inplace=True)
original_country_df.fillna(0, inplace=True)

Add variables:

In [ ]:
original_country_df['one_dose'] = original_country_df['people_vaccinated'] - original_country_df['people_fully_vaccinated']
original_country_df['one_dose_per_hundred'] = (original_country_df['people_vaccinated_per_hundred'] - original_country_df['people_fully_vaccinated_per_hundred']).round(2)
original_country_df['population'] = (original_country_df['people_fully_vaccinated']/original_country_df['people_fully_vaccinated_per_hundred'].replace(0, 1)*100).astype(int)
original_country_df['how_many_more_vaccines'] = (2*(original_country_df['population'] - original_country_df['people_vaccinated']) + original_country_df['one_dose']).astype(int)
original_country_df['days_until_done'] = (original_country_df['how_many_more_vaccines']/original_country_df['daily_vaccinations'].replace(0, 1)).round(2)
original_country_df.head()

Sort the DataFrame first by priority countries, then by total vaccinations:

In [ ]:
priority_list = ["United States", "Russia", "India", "China", "France", "United Kingdom", "Germany",
                "Japan", "Canada", "Australia"]

country_df = original_country_df.loc[original_country_df['date'] == days_ago]
country_df.sort_values("total_vaccinations", ascending=False, inplace=True)

priority_ser_list = []
for country in priority_list:
    priority_ser_list.append(country_df.loc[country_df['location'] == country].iloc[0])

priority_df = pd.DataFrame(priority_ser_list)
not_priority_df = country_df.loc[~country_df['location'].isin(priority_list)]

country_df = priority_df.append(not_priority_df)
country_df.head()

Note, some countries do not have data:

In [ ]:
not_included_list = []
for name in country_abbrev:
    code = country_abbrev[name]
    if code not in country_df['iso_code'].unique():
        not_included_list.append(name)
        
print(f"Countries without data: {not_included_list}")

<h3> Figure 1: Total Vaccinations Given in the World </h3>

Visualization of raw numbers for vaccinations given in the world. 

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=country_df['iso_code'],
    z=country_df['total_vaccinations'].astype(int),
    colorscale="Reds",
    colorbar_title="Total Vaccinations Given"
))

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Total Vaccinations Given in the World on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Figure 2: Total People Vaccinated in the World </h3>

Visualization of the total number of people vaccinated in the world. Also includes data on people fully vaccinated and people who only have one dose of a vaccine (calculated as people vaccinated minus people fully vaccinated).

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=country_df['iso_code'],
    z=country_df['people_vaccinated'].astype(int),
    colorscale="Reds",
    colorbar_title="Total People Vaccinated",
    text='Fully Vaccinated: ' + country_df['people_fully_vaccinated'].astype(int).apply('{:,}'.format) + 
        '<br>Only One Dose: ' + country_df['one_dose'].astype(int).apply('{:,}'.format)
))

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Total People Vaccinated in the World on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Figure 3: Percentage of People Vaccinated by Country </h3>

Visualization of the percentage of people vaccinated by country. Also includes data on the percentage of people fully vaccinated and the percentage of people with only one dose by country (calculated as the percentage of people vaccinated minus the percentage of people fully vaccinated).

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=country_df['iso_code'],
    z=country_df['people_vaccinated_per_hundred'].astype(float),
    colorscale="Reds",
    colorbar_title="Percentage Vaccinated",
    text='Fully Vaccinated Percentage: ' + country_df['people_fully_vaccinated_per_hundred'].apply('{:,}'.format) + '%' + 
        '<br>Only One Dose Percentage: ' + country_df['one_dose_per_hundred'].apply('{:,}'.format) + '%'
))

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Percentage of People Vaccinated by Country in the World on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Figure 3.5: Percentage of People Fully Vaccinated by Country </h3>

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=country_df['iso_code'],
    z=country_df['people_fully_vaccinated_per_hundred'].astype(float),
    colorscale="Reds",
    colorbar_title="Percentage Fully Vaccinated",
    text='Vaccinated Percentage: ' + country_df['people_vaccinated_per_hundred'].apply('{:,}'.format) + '%' + 
        '<br>Only One Dose Percentage: ' + country_df['one_dose_per_hundred'].apply('{:,}'.format) + '%'
))

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Percentage of People Fully Vaccinated by Country in the World on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Figure 4. Daily Vaccinations in the World </h3>

Visualization of the rate vaccines are being given in the world at this time.

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=country_df['iso_code'],
    z=country_df['daily_vaccinations_per_million'].astype(float),
    colorscale="Reds",
    colorbar_title="Per Million",
    text='Total Daily Vaccinations: ' + country_df['daily_vaccinations'].astype(int).apply('{:,}'.format) 
))

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Daily Vaccinations Per Million in the World on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Fig 5. Days Left Until Everyone is Fully Vaccinated in the World </h3>

Visualization of how many days are left until everyone is fully vaccinated in the world. Also includes data on how many total vaccinations are needed and the population of the country.

In [ ]:
fig = px.choropleth(
    country_df,
    locations=country_df['iso_code'],
    color='days_until_done',
    color_continuous_scale="Reds",
    labels={'days_until_done': 'Days Left'},
    range_color=(0, 1000)
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    title={'text': f"Days Left Until Fully Vaccinated in the World on {today}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}}
)

fig.show()

<h3> Figure 6. Progress by Country </h3>

Same visualizations as above except you can look at the progress/trend each country has been making. Default is to look 30 days back. Change ```days_prev``` in the ```country_visualization``` function to look a different number of days back.

In [ ]:
def country_visualization(var, title, yaxis = 'Percentage', days_prev = 30):
    c = 0
    buttons_list = []
    base_list = [False for i in range(len(country_df))]
    for i in country_df.index:
        tmp_list = base_list.copy()
        tmp_list[c] = True
        buttons_list.append(dict(
            args=[{"visible": tmp_list}],
            label=country_df.loc[i]['location'],
            method="update"
        ))
        c += 1

    c = 0
    fig = go.Figure()
    for i in country_df.index:
        location_name = country_df.loc[i]['location']
        prev_df = original_country_df.loc[original_country_df['location'] == location_name][-days_prev:]
        fig.add_trace(go.Scatter(name=location_name, x=prev_df["date"], y=prev_df[var], visible=False if c != 0 else True))
        c += 1

    fig.update_layout(
        updatemenus=[
            dict(
                buttons=buttons_list,
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )

    fig.update_layout(title={'text': f'{title} - World, Past {days_prev} Days', 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20}}, showlegend=False, yaxis_title=f"{yaxis}")
    fig.show()

In [ ]:
country_visualization('people_vaccinated_per_hundred', 'Percentage of People Vaccinated')

In [ ]:
country_visualization('people_fully_vaccinated_per_hundred', 'Percentage of People Fully Vaccinated')

In [ ]:
country_visualization('daily_vaccinations', 'Number of Daily Vaccinations', yaxis='Per Million')

In [ ]:
country_visualization('how_many_more_vaccines', 'How Many More Vaccines are Needed?', yaxis="Total Vaccines")

In [ ]:
country_visualization('days_until_done', 'Days Left Until Fully Vaccinated', yaxis="Days Left")

# Concluding Remarks

If you've read down this far in the notebook, thank you so much. This notebook took quite a long time to make, but that's aside from the point - these visualizations are for the community, and I'd like this project to also be for the community. So please leave an <span style="color: green">upvote</span> - it takes literally less than a second - so this notebook/project gains more traction & recognition.

As I say in my <a href="https://www.kaggle.com/ironicninja/daily-covid-19-interactive-plots">related COVID-19 notebook</a>, if you have any suggestions or code for other visualizations, please let me know in the comments or in DMs.

Stay safe, stay healthy, educate yourself. Thanks!